In this competition, Zillow is asking you to predict the log-error between their Zestimate and the actual sale price, given all the features of a home. The log error is defined as

                                 logerror=log(Zestimate)−log(SalePrice)
and it is recorded in the transactions file train.csv. In this competition, you are going to predict the logerror for the months in Fall 2017. Since all the real estate transactions in the U.S. are publicly available, we will close the competition (no longer accepting submissions) before the evaluation period begins.

* Train/Test split
You are provided with a full list of real estate properties in three counties (Los Angeles, Orange and Ventura, California) data in 2016.
* The train data has all the transactions before October 15, 2016, plus some of the transactions after October 15, 2016.
* The test data in the public leaderboard has the rest of the transactions between October 15 and December 31, 2016.
* The rest of the test data, which is used for calculating the private leaderboard, is all the properties in October 15, 2017, to December 15, 2017. This period is called the "sales tracking period", during which we will not be taking any submissions.
* You are asked to predict 6 time points for all properties: October 2016 (201610), November 2016 (201611), December 2016 (201612), October 2017 (201710), November 2017 (201711), and December 2017 (201712).
* Not all the properties are sold in each time period. If a property was not sold in a certain time period, that particular row will be ignored when calculating your score.
* If a property is sold multiple times within 31 days, we take the first reasonable value as the ground truth. By "reasonable", we mean if the data seems wrong, we will take the transaction that has a value that makes more sense.
* File descriptions
* properties_2016.csv - all the properties with their home features for 2016. Note: Some 2017 new properties don't have any data yet except for their parcelid's. Those data points should be populated when properties_2017.csv is available.
* properties_2017.csv - all the properties with their home features for 2017 (released on 10/2/2017)
* train_2016.csv - the training set with transactions from 1/1/2016 to 12/31/2016
* train_2017.csv - the training set with transactions from 1/1/2017 to 9/15/2017 (released on 10/2/2017)
sample_submission.csv - a sample submission file in the correct format

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
%matplotlib inline
import pandas as pd
import missingno as msno
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy.stats import ttest_ind
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.feature_selection import SelectFromModel
import h2o
from h2o.estimators import H2ORandomForestEstimator
from h2o.grid.grid_search import H2OGridSearch

#sets up pandas table display
pd.set_option('display.width', 800)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
#stop scientific notation
pd.options.display.float_format = '{:.2f}'.format

In [ ]:
# Making a list of missing value types
missing_values = ["n/a", "na", "--"]

In [ ]:
#load the 2016 properties data and target variable
house_2016_df = pd.read_csv('../input/properties_2016.csv', na_values = missing_values, low_memory=False)
house_2017_df = pd.read_csv('../input/properties_2017.csv', na_values = missing_values, low_memory=False)
house_log_2016 = pd.read_csv('../input/train_2016_v2.csv', low_memory=False)
house_log_2017 = pd.read_csv('../input/train_2017.csv', low_memory=False)

In [ ]:
#get the size
print(house_2016_df.shape)
print(house_2017_df.shape)
print(house_log_2016.shape)
print(house_log_2017.shape)

In [ ]:
house_2017_df.sample(10)

In [ ]:
#merge the trasaction dataset fro 2016-2017
house_log_full = pd.concat([house_log_2016,house_log_2017],ignore_index=True)
house_log_full.head(10)

In [ ]:
#join the 2017 train set with 2016&2017 target variable
house_2017_full = house_2017_df.merge(house_log_full, on = 'parcelid')
house_2017_full.head(5)

### Data EDA

In [ ]:
#impute boolean variables 
house_2017_full['fireplaceflag'].replace(True, 1, inplace=True)
house_2017_full['fireplaceflag'].fillna(0, inplace = True)
house_2017_full['hashottuborspa'].replace(True, 1, inplace=True)
house_2017_full['hashottuborspa'].fillna(0, inplace = True) 
house_2017_full['pooltypeid10'].fillna(0, inplace = True) 
house_2017_full['pooltypeid2'].fillna(0, inplace = True)
house_2017_full['pooltypeid7'].fillna(0, inplace = True)


In [ ]:
#plot distribution of target variable log error
from scipy.stats import zscore
house_2017_full["logerror_zscore"] = zscore(house_2017_full["logerror"])
house_2017_full["is_outlier"] = house_2017_full["logerror_zscore"].apply(
  lambda x: x <= -2.5 or x >= 2.5
)

plt.figure(figsize=(12,8))
sb.distplot(house_2017_full[~house_2017_full['is_outlier']].logerror.values, bins=50, kde=False)
plt.xlabel('logerror', fontsize=12)
plt.title('logerror distribution')
plt.show()                     

In [ ]:
#explore the missing value
missing_df = house_2017_full.isnull().sum(axis=0).reset_index()
missing_df.columns = ['column_name', 'missing_count']
missing_df = missing_df.loc[missing_df['missing_count']>0]
missing_df = missing_df.sort_values(by='missing_count')

ind = np.arange(missing_df.shape[0])
width = 0.9
fig, ax = plt.subplots(figsize=(12,18))
rects = ax.barh(ind, missing_df.missing_count.values, color='lightblue')
ax.set_yticks(ind)
ax.set_yticklabels(missing_df.column_name.values, rotation='horizontal')
ax.set_xlabel("Count of missing values")
ax.set_title("Number of missing values in each column")
plt.show() 

In [ ]:
#for continues variables, plot the scatter plot
low_na_num_features = ['lotsizesquarefeet','finishedsquarefeet12','calculatedbathnbr','fullbathcnt','roomcnt','bedroomcnt','bathroomcnt','landtaxvaluedollarcnt', 'taxamount', 'structuretaxvaluedollarcnt', 'taxvaluedollarcnt']

In [ ]:
house_2017_full[low_na_num_features].describe()

In [ ]:
#explore categorical features with low missing values
low_na_cat_features = ['airconditioningtypeid', 'decktypeid', 'heatingorsystemtypeid', 
                       'hashottuborspa', 'heatingorsystemtypeid','regionidcounty', 'regionidcity',
                       'propertycountylandusecode','propertylandusetypeid','yearbuilt','assessmentyear']

In [ ]:
#for continues variables, plot the scatter plot#correlation matrix to measure the corr between continuous variables
num_var = ['basementsqft', 'bathroomcnt', 'bedroomcnt', 'buildingqualitytypeid','buildingclasstypeid','threequarterbathnbr','calculatedfinishedsquarefeet',
           'calculatedbathnbr','finishedsquarefeet6','finishedsquarefeet12', 'finishedsquarefeet13', 'finishedsquarefeet15','finishedsquarefeet50','fireplacecnt',
           'fullbathcnt','garagecarcnt', 'garagetotalsqft','lotsizesquarefeet','poolsizesum', 'poolcnt', 'numberofstories', 'poolcnt', 'poolsizesum','roomcnt', 
           'unitcnt','yardbuildingsqft17','yearbuilt', 'yardbuildingsqft26', 'latitude', 'longitude','taxvaluedollarcnt', 'structuretaxvaluedollarcnt', 'landtaxvaluedollarcnt', 
           'taxamount','logerror']
corr = house_2017_full[num_var].corr()
ig, ax = plt.subplots(figsize=(25,15)) 
sb.heatmap(corr, cmap="YlGnBu", annot=True, ax = ax)

In [ ]:
# Drop Multicollinearity variables
house_2017_full.drop(['taxvaluedollarcnt', 'structuretaxvaluedollarcnt', 'landtaxvaluedollarcnt', 'bathroomcnt',
                      'fullbathcnt','finishedsquarefeet6', 'finishedsquarefeet12', 'finishedsquarefeet13','finishedsquarefeet15',
                      'finishedsquarefeet50'], axis=1, inplace=True)

In [ ]:
#one way ANOVA analysis between cat variables has more than two levels and logerror
for i in range(0, len(low_na_cat_features)):
    formular = str('logerror ~ '+low_na_cat_features[i])
    model = ols(formular,data = house_2017_full).fit()
    anova_result = sm.stats.anova_lm(model, typ=2)
    print (anova_result)
    print("\n")

* significant var:'airconditioningtypeid', 'decktypeid', 'heatingorsystemtypeid', 'hashottuborspa', 'heatingorsystemtypeid','regionidcounty', 'regionidcity', 'regionidneighborhood', 'storytypeid', 'typeconstructiontypeid', 'assessmentyear','yearbuilt'
* not significant var: 'architecturalstyletypeid','propertylandusetypeid'regionidzip'

In [ ]:
#t test between cat variables has two levels and taxamount
ttest_ind(house_2017_full['logerror'], house_2017_full['fireplaceflag'])

In [ ]:
# since the dataset has many categorical variables, and using sklean random forest requires one-hot encoding
# so we use h2o random forest instead
# create h2o object
h2o.init()

In [ ]:
#remove logerror outlier and other add-in variables
house_2017_temp = house_2017_full[~house_2017_full['is_outlier']]
house_2017_tree = house_2017_temp.drop(['logerror_zscore','is_outlier','parcelid'], axis =1)
# house_2017_full_hf = h2o.H2OFrame(house_2017_tree)
house_2017_tree.head(5)

In [ ]:
#track the data shape
house_2017_tree.shape

In [ ]:
#defind the model
# h2o_tree = H2ORandomForestEstimator(ntrees = 50, max_depth = 20, nfolds =10)
#train the model,if x not specify,model will use all x except the y column
# h2o_tree.train(y = 'logerror', training_frame = house_2017_full_hf)
#print variable importance
# h2o_tree_df = h2o_tree._model_json['output']['variable_importances'].as_data_frame()
#visualize the importance
"""
plt.rcdefaults()
fig, ax = plt.subplots(figsize = (10, 10))
variables = h2o_tree._model_json['output']['variable_importances']['variable']
y_pos = np.arange(len(variables))
scaled_importance = h2o_tree._model_json['output']['variable_importances']['scaled_importance']
ax.barh(y_pos, scaled_importance, align='center', color='green', ecolor='black')
ax.set_yticks(y_pos)
ax.set_yticklabels(variables)
ax.invert_yaxis()
ax.set_xlabel('Scaled Importance')
ax.set_title('Variable Importance')
plt.show()

#choose features have importance score >0.2
feature_score = 0.2
selected_features = h2o_tree_df[h2o_tree_df.scaled_importance>=feature_score]['variable']
selected_features
"""

In [ ]:
selected_features = ['transactiondate', 'regionidneighborhood','taxamount','calculatedfinishedsquarefeet'
                     ,'yearbuilt','lotsizesquarefeet','propertyzoningdesc','garagetotalsqft'
                     ,'latitude','longitude','bedroomcnt','buildingqualitytypeid'
                     ,'calculatedbathnbr','yardbuildingsqft17']

### Feature Engineering
H2O: Missing values are interpreted as containing information (i.e., missing for a reason), rather than missing at random. During tree building, split decisions for every node are found by minimizing the loss function and treating missing values as a separate category that can go either left or right. XGBoost will automatically learn which is the best direction to go when a value is missing.

### Build XGboosting model by using h2o<br>
http://docs.h2o.ai/h2o/latest-stable/h2o-docs/data-science/xgboost.html<br>
http://uc-r.github.io/regression_preparation<br>
https://dzone.com/articles/machine-learning-with-h2o-hands-on-guide-for-data<br>

In [ ]:
selected_cols = (pd.Series(selected_features)).append(pd.Series(['logerror']))
#split data to training and test data set
X_train,X_test= train_test_split(house_2017_tree[selected_cols], test_size=0.33, random_state=42)
print(X_train.shape)
print(X_test.shape)

In [ ]:
#transfer to h2o dataframe
X_train_h2o = h2o.H2OFrame(X_train)
X_test_h2o = h2o.H2OFrame(X_test)

In [ ]:
param = {
      "ntrees" : 100
    , "learn_rate" : 0.02
    , "max_depth" : 10
    , "sample_rate" : 0.7
    , "col_sample_rate_per_tree" : 0.9
    , "min_rows" : 5
    , "seed": 4241
    , "score_tree_interval": 100
    ,  'nfolds': 10
    , "stopping_metric" : "MSE"
}
from h2o.estimators import H2OXGBoostEstimator
model = H2OXGBoostEstimator(**param)
model.train(y = 'logerror', training_frame = X_train_h2o)

model evaluation: http://docs.h2o.ai/h2o/latest-stable/h2o-docs/performance-and-prediction.html#metric-best-practices-regression

In [ ]:
#print training model summary
print(model.summary)

In [ ]:
# create the new test set metrics
my_metrics = model.model_performance(test_data=X_test_h2o) 
my_metrics

### Hyper-Parameter Search

* https://xgboost.readthedocs.io/en/latest/tutorials/param_tuning.html
* https://www.dataiku.com/learn/guide/code/python/advanced-xgboost-tuning.html
* https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/

Row and column sampling (sample_rate and col_sample_rate) can improve generalization and lead to lower validation and test set errors. Good general values for large datasets are around 0.7 to 0.8 (sampling 70-80 percent of the data) for both parameters, as higher values generally improve training accuracy.<br>
col_sample_rate_per_tree: This option specifies the column sampling rate for each tree (without replacement). This can be a value from 0.0 to 1.0 and defaults to 1. Note that it is multiplicative with col_sample_rate, so setting both parameters to 0.8, for example, results in 64% of columns being considered at any given node to split.

In [ ]:
hyper_params = {'max_depth' : [4,6,8,12,16,20]
               ,"learn_rate" : [0.1, 0.01, 0.0001]
               }
param_grid = {
      "ntrees" : 50
    , "sample_rate" : 0.7
    , "col_sample_rate_per_tree" : 0.9
    , "min_rows" : 5
    , "seed": 4241
    , "score_tree_interval": 100
    ,  'nfolds': 10
    , "stopping_metric" : "MSE"
}
model_grid = H2OXGBoostEstimator(**param_grid)

In [ ]:
#Build grid search with previously made GBM and hyper parameters
#In a cartesian grid search, users specify a set of values for each hyperparamter that they want to search over, and H2O will train a model for every combination of the hyperparameter values. This means that if you have three hyperparameters and you specify 5, 10 and 2 values for each, your grid will contain a total of 5*10*2 = 100 models.
"""
grid = H2OGridSearch(model_grid,hyper_params,
                         grid_id = 'depth_grid',
                         search_criteria = {'strategy': "Cartesian"})


#Train grid search
grid.train(y='logerror',
           training_frame = X_train_h2o)
"""

In [ ]:
#print the grid search results
# Get the grid results, sorted by validation AUC
"""
xgb_gridperf = grid.get_grid(sort_by='mse', decreasing=True)
xgb_gridperf

# Grab the top xgb model, chosen by validation mse
best_xgb = xgb_gridperf.models[17]

# Now let's evaluate the model performance on a test set
# so we get an honest estimate of top model performance
best_xgb_model = best_xgb.model_performance(test_data=X_test_h2o)
#0.1         6   depth_grid_model_4  0.006894977436261656

best_xgb_model.mse()
"""

In [ ]:
#build the best model based on previous training
best_param = {
      "ntrees" : 100
    , "learn_rate" : 0.1
    , "max_depth" : 6
    , "sample_rate" : 0.7
    , "col_sample_rate_per_tree" : 0.9
    , "min_rows" : 5
    , "seed": 4241
    , "score_tree_interval": 100
    ,  'nfolds': 10
    , "stopping_metric" : "MSE"
}

best_model = H2OXGBoostEstimator(**best_param)
best_model.train(y = 'logerror', training_frame = X_train_h2o)

In [ ]:
# create the test set metrics for the best model
best_metrics = best_model.model_performance(test_data=X_test_h2o) 
best_metrics